In [89]:
# input: ascending ordered list of generators of I
# output: list of the the basis vectors
def basis_vectors(generators):
    vectors = []
    vector = []
    for i in range(len(generators)):
        vector.append(0)
    for j in range(len(generators)):
        v = list(vector)
        v[j] = 1
        vectors.append(v)
        
    key = {}
    place = 0
    for poly in generators:
        key.update({str(poly) : vectors[place]})
        place += 1
    print(key)
    
    return vectors


# input: list of the basis vectors (initial P)
# output: list of pairs of the basis vectors 
def initial_pairs(basis_vectors):
    pairs = []
    q = list(basis_vectors) # q is a list of second polynomials in the pairs to be formed, pairs = [_,q]
    for i in basis_vectors:
        # remove the i-th pol. from q to prevent repetition in pairs
        q.remove(i)
        for j in q:
            pairs.append([i, j]) #sets the initial pairs

    return pairs


# input: vector form of a pair from initial P (ex: [1, 0, 1] = e1 & e3), ascending ordered list of generators of I
# output: Koszul syzygy
def koszul(pair, generators):
    first_poly = list(pair[0])
    second_poly = list(pair[1])
    
    i = first_poly.index(1)
    j = second_poly.index(1)
    second_poly[i] = -(generators[j])
    second_poly[j] = generators[i]
    
    return second_poly


# input: a vector
# output: the signature of the vector and which basis vector it corresponds to, [sig, basis vector]
def signature(vector):
    size = len(vector)
    sig = []
    for i in range(size):
        sig.append(0)
    zero = list(sig)
    while sig == zero:
        if vector[size-1] != 0:
            if vector[size-1] in CC:
                sig[size-1] = vector[size-1]
            else:
                sig[size-1] = (vector[size-1].numerator()).lt()
        else:
            size -= 1
    return [sig, size-1]  


# input: vector, ascending ordered list of generators of I
# output: polynomial form of the vector
def poly(vector, generators):
    gen = 0
    place = 0
    for mult in vector:
        gen += mult*generators[place]
        place += 1
    return gen

# input: 2 polynomials, f, g
# output: the multipliers of S(f,g), [multiplier for f, mult for g]
def lcm_lm_(f, g):
    if f in CC and g in CC:
        return [lcm(f, g)/f, -lcm(f, g)/f]
    elif f in CC:
        return [lcm(f, (g.numerator()).lm())/f, -lcm(f, (g.numerator()).lm())/(g.numerator()).lm()]
    elif g in CC:
        return [lcm((f.numerator()).lm(), g)/(f.numerator()).lm(), -lcm((f.numerator()).lm(), g)/g]
    else:
        return [lcm((f.numerator()).lm(), (g.numerator()).lm())/(f.numerator()).lm(), -lcm((f.numerator()).lm(), (g.numerator()).lm())/(g.numerator()).lm()]

    
# input: 2 vectors, ascending ordered list of generators of I    
# output: S-vector of the inputs  
def s_vector(f_v, g_v, generators):
    s = list(f_v)
    f = poly(f_v, generators)
    g = poly(g_v, generators)
    print('vector f,g: ', f_v, g_v)
    print('f,g: ', f,g)
    m_fg = lcm_lm_(f, g)
    v_leads = []
    vector_1 = []
    vector_2 = []
    place = 0
    for i in f_v:
        print('i in f_v: ', i)
        vector_1.append(i*m_fg[0])
        vector_2.append(-1*g_v[place]*m_fg[1])
        s[place] = (i*m_fg[0]) + (g_v[place]*m_fg[1])
        place += 1
    v_leads.append(signature(vector_1))
    v_leads.append(signature(vector_2))
    
    print([s,v_leads])
    return [s, v_leads]  


# input: a vector, h, to be reduced, set of ascending ordered vectors, G, to reduce h
# output: the reduced vector
def reg_sig_red(g, G, generators):
#     print('g: ', g, 'G: ', G)
    l = len(G) - 1
    u = g
#     print('poly start g: ',poly(u, generators))
    r = 0
    while poly(u, generators) != r:
        if (poly(u, generators)-r).parent() == R:
            m = (poly(u, generators)-r).lt()
        else:            
            m = (poly(u, generators)-r).numerator().lt()
        i = 0
        reductionoccurred = False
        
        while i <= l and reductionoccurred == False:
            d = []
#             print('poly g*: ',poly(u, generators))

#             print('the poly h_i: ',poly(G[i], generators))
            if poly(G[i], generators).parent() == R:
                mult_d = m/(poly(G[i], generators).lt())
            else:
                mult_d = m/((poly(G[i], generators).numerator()).lt())
            
            for element in G[i]:
                d.append(element*mult_d)

            sig_d = signature(d)
            sig_u = signature(u)
            
            if mult_d in R and (sig_u[1] > sig_d[1] or (sig_u[1] == sig_d[1] \
            and sig_u[0][sig_u[1]] > sig_d[0][sig_d[1]])):
                place = 0
                for i in d:
                    u[place] -= i
                    place += 1
                reductionoccurred = True
            else:
                i += 1
        if reductionoccurred == False:
            r += m
#     print('u: ', u)
    return u


# input: list of vectors of Groebner basis, ascending ordered list of generators of I
# output: list of poly of Groebner basis
def G_poly(G, generators):
    basis = []
    for vector_G in G:
        gen = poly(vector_G, generators)
        basis.append(gen)
#     print('G: ', G)
    return basis

    
'''
How vectors are to be interpreted:
    each entry in the vector corresponds to what vector basis it is multiplied to
    ex: [x, y+1, 0] --> x*e1 + (y+1)*e2 + (0)*e3
'''
    
# input: list of monic polynomials that generate the ideal
# output: a Groebner basis for the ideal
def signature_alg(generators):
    G = []   # set intermediate Groebner basis to be a empty list
    P = basis_vectors(generators)
    S = []    
    pairs = initial_pairs(P)
    for pair in pairs:
        S.append(koszul(pair, generators))
        
    while P != []:
        print('G*', G)
        print('P*', P)
        g = P[0]
        del P[0]
        sig_g = signature(g)
        grp = []
#         print('g,s: ',G, S )
        for gen in G:
            grp.append(signature(gen))
        for syzygy in S:
            grp.append(signature(syzygy))
#         print('grp**: ', grp)
        grp_ = list(grp)
        for g_s in grp_:
            if g_s[0][g_s[1]] == 1 or g_s[0][g_s[1]] == -1:
                grp.remove(g_s)
#         print('grp: ', grp)
        
        # check if criterion is False
        criterion = False
        for sig_grp in grp:
            if sig_g[1] == sig_grp[1] and sig_g[0][sig_g[1]]/sig_grp[0][sig_grp[1]] in R:
                criterion = True
                break 
                
                
        print('criterion:  ',criterion)
        if criterion == False:
            h = reg_sig_red(g, G, generators)
#             print('h: ', h, poly(h, generators))
            if poly(h, generators) == 0:
                S.append(h)
                print('h reduced to 0')

            else:
                # make h have its polynomial LC = 1
                if poly(h, generators).parent() == R:
                    change = 1/(poly(h, generators).lc())
                else:
                    change = 1/((poly(h, generators).numerator()).lc())#*******
                
                place = 0
                for entry in h:
                    h[place] = entry*change
                    place += 1
                
                # update P
                for gen in G:
                    p = list(P)
                    s_v = s_vector(gen, h, generators)
                    new = s_v[0]
                    ks = s_v[1][0]
                    hs = s_v[1][1]
                                   
                    if ks != hs:   # checks if S-vector is regular
                        if p == []:
                            P.append(new)

                        if p != []:
                            last = len(p)-1
                            sig_new = signature(new) 
                            sig_p = signature(p[last])

                            add = False

                            if sig_new[1] > sig_p[1] or (sig_new[1]==sig_p[1] \
                            and sig_new[0][sig_new[1]] > sig_p[0][sig_p[1]]):
                                P.append(new)

                            else:
                                while add == False:
                                    if last == 0:
                                        P.insert(0, new)
                                        add = True
                                    else:
                                        del p[last]
                                        last -= 1
                                        sig_p = signature(p[last])

                                        if sig_new[1] > sig_p[1] or (sig_new[1]==sig_p[1] \
                                        and sig_new[0][sig_new[1]] > sig_p[0][sig_p[1]]):
                                            P.insert(last+1, new)
                                            add = True

                                        if last == 0:
                                            P.insert(0, new)
                                            add = True
                                            
                G.append(h)
#             print('p:  ', P)
    return(G_poly(G, generators))
 
'''
list of generators must be listed in ascending order
'''    
R.<t,x,y,z,w> = PolynomialRing(QQ, order = 'deglex')
# print(signature_alg([x*y - y*t, x^2-z*t, z^3 - t^3])) #WORKS
# print(signature_alg([3*x-6*y-2*z,2*x-4*y+4*w,x-2*y-z-w])) #pg. 95, WORKS
# print(signature_alg([z-t^2, y-t^3, x-t^4])) #pg. 101, G = {t^2-z, ty-z^2, tz-y, x-z^2, y^2-z^3} have last gen as y^2*z - z^4!!
print(signature_alg([x*z-y^2, x^3-z^2])) #pg.97
# print('***********:' ,[z - y^2, x^3 - z^2, x^2*y^2 - z^3, x*y^4-z^4, y^6-z^5])
# print(signature_alg([x^3-2*x*y, x^2*y-2*y^2+x])) #example pg.90, not working W/ MINIMAL, WORKS IF GO STRAIGHT TO REDUCED
# print(poly([x*y^2 + x^2*z, -z^2], [x*z-y^2, x^3-z^2]))

{'x*z - y^2': [1, 0], 'x^3 - z^2': [0, 1]}
G* []
P* [[1, 0], [0, 1]]
criterion:   False
G* [[1, 0]]
P* [[0, 1]]
criterion:   False
vector f,g:  [1, 0] [0, 1]
f,g:  x*z - y^2 x^3 - z^2
i in f_v:  1
i in f_v:  0
[[x^2, -z], [[[x^2, 0], 0], [[0, z], 1]]]
G* [[1, 0], [0, 1]]
P* [[x^2, -z]]
criterion:   False
vector f,g:  [1, 0] [-x^2, z]
f,g:  x*z - y^2 x^2*y^2 - z^3
i in f_v:  1
i in f_v:  0
[[x*y^2 + x^2*z, -z^2], [[[x*y^2, 0], 0], [[0, z^2], 1]]]
vector f,g:  [0, 1] [-x^2, z]
f,g:  x^3 - z^2 x^2*y^2 - z^3
i in f_v:  0
i in f_v:  1
[[x^3, y^2 - x*z], [[[0, y^2], 1], [[0, x*z], 1]]]
G* [[1, 0], [0, 1], [-x^2, z]]
P* [[x*y^2 + x^2*z, -z^2], [x^3, y^2 - x*z]]
criterion:   True
G* [[1, 0], [0, 1], [-x^2, z]]
P* [[x^3, y^2 - x*z]]
criterion:   False
h reduced to 0
[x*z - y^2, x^3 - z^2, x^2*y^2 - z^3]
